In [ ]:
from google.colab import files
import zipfile
uploaded = files.upload()

with zipfile.ZipFile("Emotion_Detection_Dataset.zip", "r") as zip_ref:
    zip_ref.extractall("Emotion_Detection_Dataset")

Saving Emotion_Detection_Dataset.zip to Emotion_Detection_Dataset.zip


In [ ]:
train_path = '/content/Emotion_Detection_Dataset/train'
test_path = '/content/Emotion_Detection_Dataset/test'

### efficientnet_b2_mixup_model_da.pth 24-04-2025

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import classification_report
import os
from torch.optim.lr_scheduler import CosineAnnealingLR
from timm.data import Mixup
from timm.loss import LabelSmoothingCrossEntropy
from timm.models import efficientnet_b2
import numpy as np
from timm.loss import SoftTargetCrossEntropy


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define stronger augmentations for minority classes like Disgust
def get_augmented_transform(class_name=None):
    if class_name == 'Disgust':
        return transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),  # Increased rotation range
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3),
            transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random resized crop for variation
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    else:
        return transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

# Data transforms for training and testing
train_transform = get_augmented_transform(class_name=None)  # Default transform for most classes
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=test_transform)

# Split test into val and test
val_size = len(test_dataset) // 2
test_size = len(test_dataset) - val_size
val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load pretrained EfficientNet-B2
model = efficientnet_b2(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.classifier.in_features, 7)
)
model = model.to(device)

# MixUp
mixup_fn = Mixup(mixup_alpha=0.4, cutmix_alpha=0.0, label_smoothing=0.1, num_classes=7)

# Loss function
criterion = SoftTargetCrossEntropy()

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=50)

# Evaluation function
def evaluate_model(model, loader, split_name=""):
    model.eval()
    correct = 0
    total = 0
    preds, targets = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            preds.extend(predicted.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    acc = 100 * correct / total
    print(f"\n{split_name} Accuracy: {acc:.2f}%")
    class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    print(f"\n{split_name} Classification Report:\n")
    print(classification_report(targets, preds, target_names=class_names))
    return acc, preds, targets

# Save model function
def save_model(model, optimizer, scheduler, epoch, file_path='efficientnet_b2_mixup_model_da.pth'):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'class_names': ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    }
    torch.save(checkpoint, file_path)
    print(f"Model saved to {file_path}")

# Train function with early stopping
def train_model(model, train_loader, val_loader, optimizer, scheduler, criterion, epochs=5, patience=5):
    best_val_acc = 0.0
    best_model_wts = model.state_dict()
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            if mixup_fn is not None:
                images, labels = mixup_fn(images, labels)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        val_acc, _, _ = evaluate_model(model, val_loader, split_name="Validation")

        print(f"[Epoch {epoch+1}/{epochs}] Loss: {running_loss / len(train_loader):.4f} | Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()
            save_model(model, optimizer, scheduler, epoch + 1)  # Save model
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"EarlyStopping counter: {patience_counter} out of {patience}")
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    model.load_state_dict(best_model_wts)
    return model

# Train and evaluate
model = train_model(model, train_loader, val_loader, optimizer, scheduler, criterion, epochs=50, patience=5)

train_acc, train_preds, train_labels = evaluate_model(model, train_loader, split_name="Train")
val_acc, val_preds, val_labels = evaluate_model(model, val_loader, split_name="Validation")
test_acc, test_preds, test_labels = evaluate_model(model, test_loader, split_name="Test")

Using device: cuda

Validation Accuracy: 57.93%

Validation Classification Report:

              precision    recall  f1-score   support

       Angry       0.46      0.46      0.46       494
     Disgust       0.00      0.00      0.00        59
        Fear       0.43      0.23      0.30       496
       Happy       0.76      0.88      0.82       912
         Sad       0.48      0.63      0.55       590
    Surprise       0.46      0.43      0.45       618
     Neutral       0.72      0.69      0.71       420

    accuracy                           0.58      3589
   macro avg       0.47      0.48      0.47      3589
weighted avg       0.56      0.58      0.56      3589

[Epoch 1/50] Loss: 1.6006 | Val Acc: 57.93%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model saved to efficientnet_b2_mixup_model_da.pth

Validation Accuracy: 62.69%

Validation Classification Report:

              precision    recall  f1-score   support

       Angry       0.53      0.56      0.54       494
     Disgust       1.00      0.02      0.03        59
        Fear       0.46      0.29      0.36       496
       Happy       0.84      0.88      0.86       912
         Sad       0.53      0.68      0.60       590
    Surprise       0.52      0.51      0.51       618
     Neutral       0.71      0.75      0.73       420

    accuracy                           0.63      3589
   macro avg       0.66      0.53      0.52      3589
weighted avg       0.63      0.63      0.61      3589

[Epoch 2/50] Loss: 1.4327 | Val Acc: 62.69%
Model saved to efficientnet_b2_mixup_model_da.pth

Validation Accuracy: 65.70%

Validation Classification Report:

              precision    recall  f1-score   support

       Angry       0.61      0.57      0.59       494
     Disgust       0

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

cm = confusion_matrix(train_labels, train_preds)
cm_df_t = pd.DataFrame(cm, index=class_names, columns=class_names)
cm_df_t

,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
Angry,3944,2,13,2,11,20,2
Disgust,3,428,0,0,1,1,3
Fear,16,1,4019,4,6,24,18
Happy,2,0,0,7192,7,0,7
Sad,4,0,8,8,4927,10,0
Surprise,10,0,23,2,7,4779,0
Neutral,1,3,18,5,3,0,3138


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

cm = confusion_matrix(val_labels, val_preds)
cm_df_v = pd.DataFrame(cm, index=class_names, columns=class_names)
cm_df_v

,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
Angry,320,5,54,10,44,49,12
Disgust,11,41,5,0,0,2,0
Fear,57,1,264,19,46,69,40
Happy,24,2,13,799,47,12,15
Sad,31,1,28,31,422,70,7
Surprise,49,1,83,18,100,360,7
Neutral,13,0,27,16,10,5,349


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

cm = confusion_matrix(test_labels, test_preds)
cm_df_test = pd.DataFrame(cm, index=class_names, columns=class_names)
cm_df_test

,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
Angry,287,6,44,15,49,51,12
Disgust,12,32,1,0,1,5,1
Fear,44,2,296,5,53,85,43
Happy,16,1,9,747,53,14,22
Sad,34,1,52,36,412,98,10
Surprise,49,1,78,28,97,369,7
Neutral,15,0,25,10,7,4,350
